In [27]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS as sklearn_stop_words

In [28]:
import re
import string
def clean_text(text):
    text = text.lower()
    text = text.encode('ascii', 'ignore').decode('utf-8') 
    text = re.sub(r'https?://\S+|www\.\S+', ' ', text)  #网址
    text = re.sub(r'@\S+', ' ', text)  # 艾特
    text = re.sub(r"'", ' ', text)  
    text = re.sub(r'%20', ' ', text)
    text = re.sub(r'\w*\d+\w*', '', text) # 12abc12
    text = re.sub(r' \d+ ', ' ', text)
    table = str.maketrans('','',string.punctuation) # 其他符号
    text = text.translate(table)
    tmp = text.split()
    text = " ".join(list(filter(lambda x: x not in sklearn_stop_words,tmp)))
    return text

In [29]:
train_df = pd.read_csv("train.csv")[:6000]
val_df = pd.read_csv("train.csv")[6000:]
test_df = pd.read_csv("test.csv")

train_df.keyword = train_df.keyword.fillna("")
test_df.keyword = test_df.keyword.fillna("")
val_df.keyword = val_df.keyword.fillna("")

train_df["text"] = train_df["text"] +" "+ train_df["keyword"]
test_df["text"] = test_df["text"] +" "+ test_df["keyword"]
val_df["text"] = val_df["text"] +" "+ val_df["keyword"]

train_df["text"] = train_df["text"].apply(clean_text)
test_df["text"] = test_df["text"].apply(clean_text)
val_df["text"] = val_df["text"].apply(clean_text)
train_df

,id,keyword,location,text,target
0,1,,NaN,deeds reason earthquake allah forgive,1
1,4,,NaN,forest near la ronge sask canada,1
2,5,,NaN,residents asked shelter place notified officer...,1
3,6,,NaN,people receive wildfires evacuation orders cal...,1
4,7,,NaN,just got sent photo ruby alaska smoke wildfire...,1
...,...,...,...,...,...
5995,8560,screams,BrasÌ_lia,echoes screams screams,0
5996,8561,screams,POFFIN,bts song jimin screams screams,0
5997,8562,screams,#Gladiator Û¢860Û¢757Û¢,casually phone jasmine cries screams spider sc...,0
5998,8567,screams,W.I.T.S Academy,screams screams,0


In [30]:
from gensim.models.word2vec import Word2Vec
df = pd.read_csv("train.csv")
df.keyword = df.keyword.fillna("")
df["text"] = df["text"] +" "+ df["keyword"]
df["text"] = df["text"].apply(clean_text)
token_list = [l.split() for l in df.text]
num_features = 32
min_word_count = 1
num_workers = 2
window_size = 3  # 上下文窗口大小
subsampling = 1e-3 # 高频词条降采样率

model = Word2Vec(token_list, workers=num_workers, vector_size=num_features, 
                 min_count=min_word_count,window=window_size, sample=subsampling)


# 保存已训练的模型
model_name = "twitter_32_word2vec_model"
model.save(model_name)

# # 测试模型效果

In [31]:
from keras.preprocessing.text import Tokenizer
from keras import models
from keras import layers
from keras import regularizers
from keras.preprocessing import sequence

In [32]:
max_words = 12000
embedding_dim = 32
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(train_df.text)

In [33]:
tok.index_word

{1: 's',
 2: 't',
 3: 'emergency',
 4: 'just',
 5: 'like',
 6: 'amp',
 7: 'body',
 8: 'disaster',
 9: 'burning',
 10: 'buildings',
 11: 'm',
 12: 'new',
 13: 'people',
 14: 'accident',
 15: 'news',
 16: 'nuclear',
 17: 'police',
 18: 'collapse',
 19: 'don',
 20: 'mass',
 21: 'crash',
 22: 'fires',
 23: 'video',
 24: 'attack',
 25: 'forest',
 26: 'dead',
 27: 'death',
 28: 'fatal',
 29: 'flames',
 30: 'fear',
 31: 'california',
 32: 'damage',
 33: 'man',
 34: 'day',
 35: 'floods',
 36: 'bomb',
 37: 'got',
 38: 'know',
 39: 'flood',
 40: 'storm',
 41: 'injured',
 42: 'flooding',
 43: 'time',
 44: 'going',
 45: 'debris',
 46: 'fatalities',
 47: 'evacuation',
 48: 'bags',
 49: 'oil',
 50: 'army',
 51: 'world',
 52: 'truck',
 53: 'today',
 54: 'military',
 55: 'evacuate',
 56: 'outbreak',
 57: 'love',
 58: 'derailment',
 59: 'spill',
 60: 'explosion',
 61: 'plan',
 62: 'harm',
 63: 'collided',
 64: 'deluge',
 65: 'screaming',
 66: 'destroy',
 67: 'deaths',
 68: 'bloody',
 69: 'ambulance',
 

In [34]:
model_name = "twitter_32_word2vec_model"
Word2Vec_model = Word2Vec.load(model_name)
weight = np.zeros((max_words,embedding_dim))
for i in range(1,max_words):
    weight[i] = Word2Vec_model.wv[tok.index_word.get(i,'?')]

In [35]:
train_seq = tok.texts_to_sequences(train_df.text)
val_seq = tok.texts_to_sequences(val_df.text)
test_seq = tok.texts_to_sequences(test_df.text)

train_matrix = tok.texts_to_matrix(train_df.text)
val_matrix = tok.texts_to_matrix(val_df.text)
test_matrix = tok.texts_to_matrix(test_df.text)

In [36]:
max_len = 25

In [37]:
train_seq_mat = sequence.pad_sequences(train_seq,maxlen=max_len)
test_seq_mat = sequence.pad_sequences(test_seq,maxlen=max_len)
val_seq_mat = sequence.pad_sequences(val_seq,maxlen=max_len)

## DNN

In [38]:
model = models.Sequential()
model.add(layers.Dense(32, activation='relu',input_shape=(max_words,)))
model.add(layers.Dense(8, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
             loss = 'binary_crossentropy',
             metrics=["accuracy"])
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 32)                384032    
_________________________________________________________________
dense_13 (Dense)             (None, 8)                 264       
_________________________________________________________________
dense_14 (Dense)             (None, 1)                 9         
Total params: 384,305
Trainable params: 384,305
Non-trainable params: 0
_________________________________________________________________


In [39]:
history_dense = model.fit(train_matrix,
                    train_df.target,
                   epochs=20,
                   batch_size = 128,
                   validation_data =(val_matrix,val_df.target))

Train on 6000 samples, validate on 1613 samples
Epoch 1/20
6000/6000 [==============================] - 1s 165us/step - loss: 0.6486 - accuracy: 0.7000 - val_loss: 0.6158 - val_accuracy: 0.7526
Epoch 2/20
6000/6000 [==============================] - 1s 110us/step - loss: 0.5054 - accuracy: 0.8412 - val_loss: 0.5348 - val_accuracy: 0.7855
Epoch 3/20
6000/6000 [==============================] - 1s 108us/step - loss: 0.3803 - accuracy: 0.8720 - val_loss: 0.4890 - val_accuracy: 0.7880
Epoch 4/20
6000/6000 [==============================] - 1s 111us/step - loss: 0.2989 - accuracy: 0.8948 - val_loss: 0.4816 - val_accuracy: 0.7799
Epoch 5/20
6000/6000 [==============================] - 1s 108us/step - loss: 0.2470 - accuracy: 0.9122 - val_loss: 0.4959 - val_accuracy: 0.7657
Epoch 6/20
6000/6000 [==============================] - 1s 107us/step - loss: 0.2084 - accuracy: 0.9275 - val_loss: 0.5287 - val_accuracy: 0.7570
Epoch 7/20
6000/6000 [==============================] - 1s 110us/step - loss

## RNN

In [40]:
model = models.Sequential()
model.add(layers.Embedding(input_dim=max_words, output_dim=embedding_dim,
                           input_length = max_len))

model.add(layers.SimpleRNN(32))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
             loss = 'binary_crossentropy',
             metrics=["accuracy"])
model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 25, 32)            384000    
_________________________________________________________________
simple_rnn_3 (SimpleRNN)     (None, 32)                2080      
_________________________________________________________________
dense_15 (Dense)             (None, 1)                 33        
Total params: 386,113
Trainable params: 386,113
Non-trainable params: 0
_________________________________________________________________


In [41]:
history_rnn = model.fit(train_seq_mat,
                    train_df.target,
                   epochs=20,
                   batch_size = 128,
                   validation_data =(val_seq_mat,val_df.target))

D:\school\aconda\envs\tensorflow\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 6000 samples, validate on 1613 samples
Epoch 1/20
6000/6000 [==============================] - 1s 144us/step - loss: 0.6448 - accuracy: 0.6340 - val_loss: 0.6584 - val_accuracy: 0.6144
Epoch 2/20
6000/6000 [==============================] - 1s 105us/step - loss: 0.4678 - accuracy: 0.8215 - val_loss: 0.6547 - val_accuracy: 0.5828
Epoch 3/20
6000/6000 [==============================] - 1s 104us/step - loss: 0.2827 - accuracy: 0.9045 - val_loss: 0.6603 - val_accuracy: 0.6683
Epoch 4/20
6000/6000 [==============================] - 1s 106us/step - loss: 0.1783 - accuracy: 0.9432 - val_loss: 0.6437 - val_accuracy: 0.6844
Epoch 5/20
6000/6000 [==============================] - 1s 104us/step - loss: 0.1248 - accuracy: 0.9600 - val_loss: 0.6869 - val_accuracy: 0.6702
Epoch 6/20
6000/6000 [==============================] - 1s 107us/step - loss: 0.0996 - accuracy: 0.9657 - val_loss: 0.7402 - val_accuracy: 0.6355
Epoch 7/20
6000/6000 [==============================] - 1s 108us/step - loss

In [42]:
model = models.Sequential()
model.add(layers.Embedding(input_dim=max_words, output_dim=embedding_dim,
                           input_length = max_len,
                           weights=[weight],
                           trainable=False))

model.add(layers.SimpleRNN(32))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
             loss = 'binary_crossentropy',
             metrics=["accuracy"])
model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 25, 32)            384000    
_________________________________________________________________
simple_rnn_4 (SimpleRNN)     (None, 32)                2080      
_________________________________________________________________
dense_16 (Dense)             (None, 1)                 33        
Total params: 386,113
Trainable params: 2,113
Non-trainable params: 384,000
_________________________________________________________________


In [43]:
history_rnn_pretrain = model.fit(train_seq_mat,
                    train_df.target,
                   epochs=20,
                   batch_size = 128,
                   validation_data =(val_seq_mat,val_df.target))

Train on 6000 samples, validate on 1613 samples
Epoch 1/20
6000/6000 [==============================] - 1s 130us/step - loss: 0.6685 - accuracy: 0.5960 - val_loss: 0.6579 - val_accuracy: 0.6181
Epoch 2/20
6000/6000 [==============================] - 1s 95us/step - loss: 0.6529 - accuracy: 0.6190 - val_loss: 0.6463 - val_accuracy: 0.6293
Epoch 3/20
6000/6000 [==============================] - 1s 95us/step - loss: 0.6455 - accuracy: 0.6320 - val_loss: 0.6359 - val_accuracy: 0.6485
Epoch 4/20
6000/6000 [==============================] - 1s 94us/step - loss: 0.6393 - accuracy: 0.6428 - val_loss: 0.6386 - val_accuracy: 0.6336
Epoch 5/20
6000/6000 [==============================] - 1s 94us/step - loss: 0.6344 - accuracy: 0.6460 - val_loss: 0.6531 - val_accuracy: 0.6330
Epoch 6/20
6000/6000 [==============================] - 1s 93us/step - loss: 0.6269 - accuracy: 0.6582 - val_loss: 0.6197 - val_accuracy: 0.6720
Epoch 7/20
6000/6000 [==============================] - 1s 94us/step - loss: 0.62

## LSTM

In [77]:
model = models.Sequential()
model.add(layers.Embedding(input_dim=max_words, output_dim=embedding_dim,
                           input_length = max_len))


model.add(layers.LSTM(32))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
             loss = 'binary_crossentropy',
             metrics=["accuracy"])
model.summary()

Model: "sequential_27"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_25 (Embedding)     (None, 25, 32)            384000    
_________________________________________________________________
lstm_14 (LSTM)               (None, 32)                8320      
_________________________________________________________________
dense_46 (Dense)             (None, 1)                 33        
Total params: 392,353
Trainable params: 392,353
Non-trainable params: 0
_________________________________________________________________


In [78]:
history_lstm = model.fit(train_seq_mat,
                    train_df.target,
                   epochs=20,
                   batch_size = 128,
                   validation_data =(val_seq_mat,val_df.target))

D:\school\aconda\envs\tensorflow\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 6000 samples, validate on 1613 samples
Epoch 1/20
6000/6000 [==============================] - 1s 203us/step - loss: 0.6379 - accuracy: 0.6275 - val_loss: 0.5973 - val_accuracy: 0.7154
Epoch 2/20
6000/6000 [==============================] - 1s 153us/step - loss: 0.4624 - accuracy: 0.8150 - val_loss: 0.5090 - val_accuracy: 0.7756
Epoch 3/20
6000/6000 [==============================] - 1s 147us/step - loss: 0.3595 - accuracy: 0.8572 - val_loss: 0.4978 - val_accuracy: 0.7874
Epoch 4/20
6000/6000 [==============================] - 1s 151us/step - loss: 0.2942 - accuracy: 0.8828 - val_loss: 0.4847 - val_accuracy: 0.7830
Epoch 5/20
6000/6000 [==============================] - 1s 147us/step - loss: 0.2461 - accuracy: 0.9047 - val_loss: 0.5212 - val_accuracy: 0.7737
Epoch 6/20
6000/6000 [==============================] - 1s 147us/step - loss: 0.2177 - accuracy: 0.9172 - val_loss: 0.5527 - val_accuracy: 0.7644
Epoch 7/20
6000/6000 [==============================] - 1s 149us/step - loss

In [59]:
model = models.Sequential()
model.add(layers.Embedding(input_dim=max_words, output_dim=embedding_dim,
                           input_length = max_len,
                          weights = [weight],
                          trainable=False))


model.add(layers.LSTM(32))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
             loss = 'binary_crossentropy',
             metrics=["accuracy"])
model.summary()

Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_16 (Embedding)     (None, 25, 32)            384000    
_________________________________________________________________
lstm_10 (LSTM)               (None, 25, 16)            3136      
_________________________________________________________________
lstm_11 (LSTM)               (None, 16)                2112      
_________________________________________________________________
dropout_1 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_28 (Dense)             (None, 16)                272       
_________________________________________________________________
dense_29 (Dense)             (None, 1)                 17        
Total params: 389,537
Trainable params: 5,537
Non-trainable params: 384,000
___________________________________________

In [60]:
history_lstm_pretrain = model.fit(train_seq_mat,
                    train_df.target,
                   epochs=20,
                   batch_size = 128,
                   validation_data =(val_seq_mat,val_df.target))

Train on 6000 samples, validate on 1613 samples
Epoch 1/20
6000/6000 [==============================] - 2s 372us/step - loss: 0.6833 - accuracy: 0.5728 - val_loss: 0.6912 - val_accuracy: 0.5325
Epoch 2/20
6000/6000 [==============================] - 2s 269us/step - loss: 0.6682 - accuracy: 0.5885 - val_loss: 0.6555 - val_accuracy: 0.6262
Epoch 3/20
6000/6000 [==============================] - 2s 264us/step - loss: 0.6498 - accuracy: 0.6253 - val_loss: 0.6333 - val_accuracy: 0.6510
Epoch 4/20
6000/6000 [==============================] - 2s 265us/step - loss: 0.6360 - accuracy: 0.6465 - val_loss: 0.6208 - val_accuracy: 0.6813
Epoch 5/20
6000/6000 [==============================] - 2s 264us/step - loss: 0.6287 - accuracy: 0.6567 - val_loss: 0.6112 - val_accuracy: 0.6720
Epoch 6/20
6000/6000 [==============================] - 2s 269us/step - loss: 0.6278 - accuracy: 0.6588 - val_loss: 0.6158 - val_accuracy: 0.6900
Epoch 7/20
6000/6000 [==============================] - 2s 267us/step - loss

## CNN

In [70]:
model = models.Sequential()
model.add(layers.Embedding(input_dim=max_words, output_dim=embedding_dim,
                           input_length = max_len))

model.add(layers.Conv1D(32, 3, activation='relu'))
model.add(layers.MaxPooling1D(3))
model.add(layers.Flatten())
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
             loss = 'binary_crossentropy',
             metrics=["accuracy"])
model.summary()

Model: "sequential_24"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_22 (Embedding)     (None, 25, 32)            384000    
_________________________________________________________________
conv1d_13 (Conv1D)           (None, 23, 32)            3104      
_________________________________________________________________
max_pooling1d_10 (MaxPooling (None, 7, 32)             0         
_________________________________________________________________
flatten_9 (Flatten)          (None, 224)               0         
_________________________________________________________________
dense_40 (Dense)             (None, 32)                7200      
_________________________________________________________________
dense_41 (Dense)             (None, 1)                 33        
Total params: 394,337
Trainable params: 394,337
Non-trainable params: 0
_______________________________________________

In [71]:
history_cnn = model.fit(train_seq_mat,
                    train_df.target,
                   epochs=20,
                   batch_size = 128,
                   validation_data =(val_seq_mat,val_df.target))

D:\school\aconda\envs\tensorflow\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 6000 samples, validate on 1613 samples
Epoch 1/20
6000/6000 [==============================] - 0s 66us/step - loss: 0.6656 - accuracy: 0.6020 - val_loss: 0.6800 - val_accuracy: 0.5518
Epoch 2/20
6000/6000 [==============================] - 0s 40us/step - loss: 0.5296 - accuracy: 0.7778 - val_loss: 0.5380 - val_accuracy: 0.7508
Epoch 3/20
6000/6000 [==============================] - 0s 41us/step - loss: 0.3653 - accuracy: 0.8603 - val_loss: 0.5177 - val_accuracy: 0.7539
Epoch 4/20
6000/6000 [==============================] - 0s 40us/step - loss: 0.2836 - accuracy: 0.8910 - val_loss: 0.5316 - val_accuracy: 0.7508
Epoch 5/20
6000/6000 [==============================] - 0s 40us/step - loss: 0.2301 - accuracy: 0.9142 - val_loss: 0.5732 - val_accuracy: 0.7421
Epoch 6/20
6000/6000 [==============================] - 0s 42us/step - loss: 0.1900 - accuracy: 0.9288 - val_loss: 0.6340 - val_accuracy: 0.7316
Epoch 7/20
6000/6000 [==============================] - 0s 39us/step - loss: 0.159

In [50]:
model = models.Sequential()
model.add(layers.Embedding(input_dim=max_words, output_dim=embedding_dim,
                           input_length = max_len,
                          weights = [weight],
                          trainable=False))

model.add(layers.Conv1D(32, 3, activation='relu'))
model.add(layers.MaxPooling1D(3))
model.add(layers.Flatten())
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
             loss = 'binary_crossentropy',
             metrics=["accuracy"])
model.summary()

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_12 (Embedding)     (None, 25, 32)            384000    
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 23, 32)            3104      
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 7, 32)             0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 224)               0         
_________________________________________________________________
dense_21 (Dense)             (None, 32)                7200      
_________________________________________________________________
dense_22 (Dense)             (None, 1)                 33        
Total params: 394,337
Trainable params: 10,337
Non-trainable params: 384,000
__________________________________________

In [51]:
history_cnn_pretrain = model.fit(train_seq_mat,
                    train_df.target,
                   epochs=20,
                   batch_size = 128,
                   validation_data =(val_seq_mat,val_df.target))

Train on 6000 samples, validate on 1613 samples
Epoch 1/20
6000/6000 [==============================] - 0s 57us/step - loss: 0.6712 - accuracy: 0.5893 - val_loss: 0.6799 - val_accuracy: 0.5511
Epoch 2/20
6000/6000 [==============================] - 0s 32us/step - loss: 0.6546 - accuracy: 0.6142 - val_loss: 0.6586 - val_accuracy: 0.5964
Epoch 3/20
6000/6000 [==============================] - 0s 36us/step - loss: 0.6443 - accuracy: 0.6292 - val_loss: 0.6580 - val_accuracy: 0.5989
Epoch 4/20
6000/6000 [==============================] - 0s 34us/step - loss: 0.6370 - accuracy: 0.6452 - val_loss: 0.6379 - val_accuracy: 0.6510
Epoch 5/20
6000/6000 [==============================] - 0s 33us/step - loss: 0.6316 - accuracy: 0.6523 - val_loss: 0.6268 - val_accuracy: 0.6764
Epoch 6/20
6000/6000 [==============================] - 0s 34us/step - loss: 0.6273 - accuracy: 0.6610 - val_loss: 0.6219 - val_accuracy: 0.6789
Epoch 7/20
6000/6000 [==============================] - 0s 32us/step - loss: 0.622

In [52]:
np.save('history_dense.npy', history_dense.history)

np.save('history_rnn.npy', history_rnn.history)
np.save('history_rnn_pretrain.npy', history_rnn_pretrain.history)


np.save('history_lstm.npy', history_lstm.history)
np.save('history_lstm_pretrain.npy', history_lstm_pretrain.history)

np.save('history_cnn.npy', history_cnn.history)
np.save('history_cnn_pretrain.npy', history_cnn_pretrain.history)